In [12]:
class Pool:
    def __init__(self, rD, rF):
        self.rD = rD
        self.rF = rF
        self.sD = rD
        self.sF = rF
        # self.limitrF = rF
        
    def getNetPrice(self):
        return self.sD/self.sF
    
    # def fillLiquidity(self):
    #     net = self.getNet()
    #     rD = self.limitrF*net
    #     if rD < self.sD:
    #         self.rF = self.limitrF
    #         self.rD = rD
    #         # print('fill liquidity success')
    #     else:
    #         self.rF = self.sD/net
    #         self.rD = self.sD
    #         # print('fill liquidity failed')
        
    def buyF(self, n):
        # assert n > 0
        # assert self.rF > n
        if self.rF > n > 0:
            # exchange
            K = self.rD*self.rF
            self.rF -= n
            rD = K/self.rF
            incomeD = rD - self.rD
            # supply change
            self.sD += incomeD
            self.sF += (n*0.9985) # 0.15% fee
            # net asset change
            # [1]repair by reduce rD
            net = self.sD/self.sF
            self.rD = self.rF * net
            # [2]do not repair
            # self.rD = rD
            return True
        else:
            return False
    
    def sellF(self, n):
        # assert n > 0
        # assert self.sF > n # 可能市面上都没有足够的F了 
        # 如果该池铸出的流动性全部收回了，此处应当如何处理？
        if self.sF > n >0:
            # exchange
            K = self.rD*self.rF
            rF = self.rF + n
            rD = K/rF
            outD = self.rD - rD
            # supply change
            self.sD -= (outD*0.9985) # 0.15% fee
            self.sF -= n
            # net asset change TODO--
            net = self.sD/self.sF
            # [1] repair by raise rD first？
            # if rF * net < self.sD:
            #     self.rF = rF
            #     self.rD = rF * net
            # else:
            #     # repair by reduce rF
            #     self.rD = rD
            #     self.rF = rD / net
            # [2] only repair by reduce rF
            self.rD = rD
            self.rF = rD / net
            return True
        else:
            return False
    
    # def sell_avg_price(self, n):
    #     if self.sF > n > 0:
    #         K = self.rD*self.rF
    #         rF = self.rF + n
    #         rD = K/rF
    #         outD = self.rD - rD
    #         return outD*0.9985
    #     else:
    #         return 0

In [19]:
AFpool = Pool(1000, 1000)
BFPool = Pool(1000, 1000)

# 加如市面上有无限供应的A=0.8B
# 1. 卖出A买入F
# 2. 卖出F买入B
# 3. 2:1
while AFpool.getNetPrice()/BFPool.getNetPrice() < 1.1:
        AFpool.buyF(1)
        if not BFPool.sellF(1):
            break

In [20]:
AFpool.sF, AFpool.sD, AFpool.rF, AFpool.rD, AFpool.getNetPrice()

(1997.5014999999246,
 2007.7749058673164,
 1,
 1.005143127986333,
 1.005143127986333)

In [21]:
BFPool.sF, BFPool.sD, BFPool.rF, BFPool.rD, BFPool.getNetPrice()

(1,
 2.444809948479451,
 0.3870359301370557,
 0.9462292924180715,
 2.444809948479451)

In [ ]:
# 如果是由市场主动构成呢？风险对冲工具。